## adjust countries to be the same information

In [104]:
%pip install geopy translate joblib scikit-learn

                                              0.0/8.3 MB ? eta -:--:--
                                              0.1/8.3 MB 2.3 MB/s eta 0:00:04
     -                                        0.2/8.3 MB 3.0 MB/s eta 0:00:03
     --                                       0.5/8.3 MB 4.2 MB/s eta 0:00:02
     ----                                     0.9/8.3 MB 5.4 MB/s eta 0:00:02
     ------                                   1.4/8.3 MB 6.5 MB/s eta 0:00:02
     ---------                                2.0/8.3 MB 7.3 MB/s eta 0:00:01
     -----------                              2.4/8.3 MB 7.6 MB/s eta 0:00:01
     -------------                            2.9/8.3 MB 7.9 MB/s eta 0:00:01
     ---------------                          3.2/8.3 MB 7.9 MB/s eta 0:00:01
     -----------------                        3.6/8.3 MB 7.9 MB/s eta 0:00:01
     -------------------                      4.0/8.3 MB 8.0 MB/s eta 0:00:01
     ---------------------                    4.5/8.3 MB 8.3 MB/s eta 0

In [105]:
%pip freeze > ..//requirements.txt

Note: you may need to restart the kernel to use updated packages.


### Reading files automatically

In [2]:
import os
import pandas as pd

def _adjust_name(name):
    if not('df_' in name):
        return 'df_' + name
    return name
def _get_files_from(base_path = '..\\data\\processed', extension='csv'):
    return [(_adjust_name(name.split('.')[0].lower()), str(os.path.join(root, name))) for (root, _, files) in os.walk(base_path, topdown=False) for name in files if (extension in name)]

def read_files(_from='..\\data\\processed', extension='csv'):
    vars = []
    for item in _get_files_from(_from, extension=extension):
        exec(f'vars.append("{item[0]}")')
        exec(f"{item[0]} = pd.read_csv(r'{item[1]}', sep=';', decimal=',')")
    return vars

# _get_files_from()

In [3]:
df_noaa_global = pd.read_csv(r'..\\data\\processed\\noaa_global\\noaa_global.csv', sep=';', decimal=',')
df_vinhos = pd.read_csv(r'..\\data\\processed\\tech_challenge\\df_vinhos.csv', sep=';', decimal=',')
df_temperature_change_data = pd.read_csv(r'..\\data\\processed\\temp_change\\temperature_change_Data.csv', sep=';', decimal=',')
df_wbpy = pd.read_csv(r'..\\data\\processed\\wbpy\\wbpy.csv', sep=';', decimal=',')

In [4]:
df_noaa_global.head(2)

,station_id,year,stat,value_min,value_mean,value_median,value_max,latitude,longitude,elevation,name
0,ITE00100554,1763,TMAX,-39,147.873973,147.0,309,45.4717,9.1892,150.0,MILAN
1,ITE00100554,1763,TMIN,-63,100.657534,99.0,233,45.4717,9.1892,150.0,MILAN


In [5]:
df_vinhos.head(2)

,pais,ano,quantidade_exportada_pais,valor_exportado_pais,id_produto,produto,quantidade_prod_rs,quantidade_com_rs,country_code
0,Afeganistão,1970,0,0,vinho de mesa,vinho de mesa,217208604.0,98327606.0,af
1,Afeganistão,1970,0,0,vm_tinto,tinto,174224052.0,83300735.0,af


### Adjust df_vinhos pais to country code

### Adjust noaa name to country code

In [6]:
import joblib

base_path = '..\\data\\processed'
dict_latlong = joblib.load(base_path+"\\dict_latlong.pkl")

In [7]:
list_latlong =[f"{_tuple.latitude},{_tuple.longitude}" for _tuple in df_noaa_global[['latitude','longitude']].drop_duplicates().itertuples()]
len(list_latlong)

112035

In [8]:
list_dict = list(dict_latlong.keys())
len(list_dict)

111870

In [9]:
list_diferenca = (list(set(list_latlong) -set(list_dict)))
list_diferenca.sort()
len(list_diferenca)

165

## Load data and adjust temperature

In [1]:
import os
import pandas as pd
import joblib
import numpy as np

base_path = '..\\data\\processed'
dict_latlong = joblib.load(base_path+"\\dict_latlong.pkl")
dict_translates = joblib.load(base_path+"\\dict_translates.pkl")

df_noaa_global = pd.read_csv(r'..\\data\\processed\\noaa_global\\noaa_global.csv', sep=';', decimal=',')
df_vinhos = pd.read_csv(r'..\\data\\processed\\tech_challenge\\df_vinhos.csv', sep=';', decimal=',')
df_temperature_change_data = pd.read_csv(r'..\\data\\processed\\temp_change\\temperature_change_Data.csv', sep=';', decimal=',')
df_wbpy = pd.read_csv(r'..\\data\\processed\\wbpy\\wbpy.csv', sep=';', decimal=',')

In [2]:
df_vinhos.ano.min()

1970

In [3]:
df_vinhos['country_code'] = [dict_translates[x].upper() for x in df_vinhos.pais]

df_noaa_global['lat_long'] = [f"{x.latitude},{x.longitude}" for x in df_noaa_global.itertuples()]
df_noaa_global = df_noaa_global.loc[df_noaa_global['lat_long'].isin(list(dict_latlong.keys()))]
df_noaa_global = df_noaa_global.loc[df_noaa_global['stat'].isin(['TMAX','TMIN','TAVG','PRCP'])]
df_noaa_global = df_noaa_global.loc[df_noaa_global['year'] > 1970]
df_noaa_global['country_code'] = [dict_latlong[f"{x.latitude},{x.longitude}"].upper() for x in df_noaa_global.itertuples()]
df_noaa_global = df_noaa_global.loc[df_noaa_global['country_code'].isin(list(df_vinhos.country_code.unique()))]

In [102]:
def _adjust_temperature(row, var):
    """
    vamos transformar dados em farenheidt para celsius;
    caso seja menor que -90 (menor temperatura ja encontrada no planeta, na antartida) 
    ou maior que 57 (maior temp, na california (death valley, usa)), retorna np.nan
    """
    if (row['stat'] in ['TMAX','TMIN','TAVG']):
        celsius = (row[var] - 32) * 5/9
        if ((celsius > -90) & (celsius < 57)):
            return celsius
        else:
            return np.nan
    return row[var]

for var in ['value_min', 'value_mean', 'value_median','value_max']:
    df_noaa_global[var] = df_noaa_global.apply(lambda row: _adjust_temperature(row, var), axis=1)

In [5]:
# imputar media no lugar do nan
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = imputer.fit(df_noaa_global[['value_min', 'value_mean', 'value_median','value_max']])
  
df_noaa_global[['value_min', 'value_mean', 'value_median','value_max']] = imputer.transform(df_noaa_global[['value_min', 'value_mean', 'value_median','value_max']])

In [6]:
df_noaa_global\
    .groupby(['year','stat','country_code'])\
    .agg({
        'value_min':('min','mean','median','max')
    })\
    .groupby(['stat'])\
    .describe()

value_min                                                                 
           min                                                                 
         count       mean          std      min     25%   50%    75%     max   
stat                                                                           
PRCP    4549.0 -13.956474   612.847574 -30866.0    0.00   0.0    0.0  4361.0  \
TAVG    4436.0   5.106177   505.333805 -29801.0 -102.25  52.5  174.0   293.0   
TMAX    4565.0  -2.686309  1059.751072 -32736.0  -71.00  76.0  219.0   350.0   
TMIN    4563.0 -91.720798   850.327390 -32756.0 -172.00 -36.0  100.0   265.0   

                          ...                                        
        mean              ...   median             max               
       count        mean  ...      75%     max   count        mean   
stat                      ...                                        
PRCP  4549.0    4.465069  ...    0.000  4361.0  4549.0   45.032095  \
TAVG  4436.0   82.863644  ...  214.125   293.0  4436.0  154.592876   
TMAX  4565.0  117.118957  ...  250.000   351.0  4565.0  195.549179   
TMIN  4563.0   23.837893  ...  150.000   270.0  4563.0  114.144642   

                                                       
                                                       
             std    min    25%    50%    75%      max  
stat                                                   
PRCP  331.024009    0.0    0.0    0.0    0.0   9271.0  
TAVG  424.243144 -324.0   69.0  198.0  243.0  27137.0  
TMAX  216.725843 -203.0  102.0  241.0  285.0   9990.0  
TMIN  434.511451 -267.0   13.0  134.0  200.5  27137.0  

[4 rows x 32 columns]

In [86]:
df_noaa_global\
    .groupby(['year','stat','country_code'])\
    .agg({
        'value_mean':('mean')
    })\
    .groupby(['stat'])\
    .describe()

value_mean                                                           
          count       mean         std        min        25%        50%   
stat                                                                      
PRCP     4549.0  56.040772  133.335521   0.000000  20.304621  32.565068  \
TAVG     1955.0  29.295964   19.049174 -55.484018  18.608652  35.391439   
TMAX     1638.0  33.804528   16.658031 -29.777778  24.180306  36.793634   
TMIN     2648.0  18.924296   21.171263 -54.512463   4.743791  22.163609   

                              
            75%          max  
stat                          
PRCP  59.945622  4361.000000  
TAVG  42.692467    56.889645  
TMAX  47.286344    56.879756  
TMIN  33.873764    56.937248

In [98]:
df_noaa_global\
     .query('country_code=="BR"')\
     .groupby(['stat','country_code'])\
     [['value_min', 'value_mean','value_max']]\
     .describe()

value_min                                               
                      count       mean        std        min        25%   
stat country_code                                                         
PRCP BR             96827.0   0.326613  29.725595   0.000000   0.000000  \
TAVG BR               617.0  36.422654  14.320039  -6.111111  25.555556   
TMAX BR                56.0  47.797619   6.726136  32.222222  43.055556   
TMIN BR               917.0  27.322792  22.571117 -56.666667  14.444444   

                                                     value_mean              
                         50%        75%          max      count       mean   
stat country_code                                                            
PRCP BR             0.000000   0.000000  8880.000000    96827.0  38.969557  \
TAVG BR            39.444444  48.888889    56.666667        8.0  52.717014   
TMAX BR            49.166667  54.027778    56.666667        0.0        NaN   
TMIN BR            33.888889  44.444444    56.666667      180.0  41.809887   

                   ...                         value_max               
                   ...        75%          max     count        mean   
stat country_code  ...                                                 
PRCP BR            ...  46.789730  8880.000000   96827.0  824.395551  \
TAVG BR            ...  54.930556    56.666667       3.0   51.851852   
TMAX BR            ...        NaN          NaN       0.0         NaN   
TMIN BR            ...  52.083333    56.944444     100.0   38.111111   

                                                                               
                          std        min         25%         50%         75%   
stat country_code                                                              
PRCP BR            362.748694   0.000000  620.000000  780.000000  968.000000  \
TAVG BR              4.490502  47.777778   49.444444   51.111111   53.888889   
TMAX BR                   NaN        NaN         NaN         NaN         NaN   
TMIN BR             15.317238 -17.777778   26.666667   43.333333   50.000000   

                                
                           max  
stat country_code               
PRCP BR            9990.000000  
TAVG BR              56.666667  
TMAX BR                    NaN  
TMIN BR              56.666667  

[4 rows x 24 columns]

In [7]:
dest_path = '..\\data\\processed\\noaa_global\\'
df_noaa2 = df_noaa_global\
    .groupby(['year','stat','country_code'])\
    .agg({
        'value_min':'min'
        , 'value_mean':'mean'
        , 'value_median':'median'
        , 'value_max':'max'
    })\
    .reset_index()
df_noaa2\
    .to_csv(dest_path+'noaa_global_final.csv', index=False, sep=';', decimal=',')

In [80]:
dest_path = '..\\data\\processed\\tech_challenge\\'
df_vinhos.to_csv(dest_path+'df_vinhos.csv', index=False, sep=';', decimal=',')
